## Non-Neural Approach


In [1]:
import numpy as np
import pandas as pd

language_path_train = "./task0-data/DEVELOPMENT-LANGUAGES/niger-congo/swa.trn"

f = open(language_path_train,encoding="utf-8")

lemmas = []
form = []
morph = []

for lines in f.readlines():
    a, b, c = lines.split("\t")
    lemmas.append(a)
    form.append(b)
    morph.append(c.split(";")[:-2])

def distance(str1, str2):
    """Simple Levenshtein implementation for eval."""
    m = np.zeros([len(str2)+1, len(str1)+1])
    for x in range(1, len(str2) + 1):
        m[x][0] = m[x-1][0] + 1
    for y in range(1, len(str1) + 1):
        m[0][y] = m[0][y-1] + 1
    for x in range(1, len(str2) + 1):
        for y in range(1, len(str1) + 1):
            if str1[y-1] == str2[x-1]:
                dg = 0
            else:
                dg = 1
            m[x][y] = min(m[x-1][y] + 1, m[x][y-1] + 1, m[x-1][y-1] + dg)
    return int(m[len(str2)][len(str1)])

In [2]:
import string

def make_alphabet(dictionnary):
    """From a list of words, create an empirical dictionnary of words"""
    scores = {}
    for word in dictionnary:
        for letter in word:
            if letter in scores.keys():
                scores[letter] += 1
            elif letter not in string.punctuation:
                scores[letter] = 1
    for v in scores.keys():
        scores[v] /= len(scores)
    
    return scores
           
            

def word_score(word,alphabet_scores):
    score = 0
    for letter in word:
        try:
            score += alphabet_scores[letter]
        except:
            continue
    return score


# Function to find the longest common substring
def LCS(X, Y):
    m = len(X)
    n = len(Y)
    maxLength = 0 
    endingIndex = m
    lookup = [[0 for x in range(n + 1)] for y in range(m + 1)]
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if X[i - 1] == Y[j - 1]:
                lookup[i][j] = lookup[i - 1][j - 1] + 1
                if lookup[i][j] > maxLength:
                    maxLength = lookup[i][j]
                    endingIndex = i
    return X[endingIndex - maxLength: endingIndex]

In [3]:
# create list containing evey morph attribute but our language
transformed = []
radical = []
lemmas, form, morph = [lemmas, form, morph]
alphabet = make_alphabet(form)
for i in range(len(lemmas)):
    sub = LCS(lemmas[i],form[i])
    transformed.append(form[i].replace(sub,"[word]"))
    radical.append(lemmas[i].replace(sub,"[word]"))

morph_list = set([b for array in morph for b in array]) 

In [4]:
morph_list

{'COND', 'DEF', 'FIN', 'HAB', 'IND', 'INDF', 'PL', 'PRF', 'SG', 'V'}

In [5]:
len(transformed), len(form), len(morph)

(3374, 3374, 3374)

Buildling dataset

In [6]:
from tqdm.notebook import tqdm


# df = pd.DataFrame(columns=list(morph_list)+["form"])
# complete matrix
M = []

# Every index in size(all_words)
for i in tqdm(range(len(transformed))):
    # new array
    morph_attributes = []
    # morph_choice = 'V' in morph_properies = {'V', 'I', '1', '2', 'J', 'O', 'T', 'G', '3', 'D', 'L', 'S', 'N', 'C', 'A', 'M', 'P'}
    for morph_choice in morph_list:
        # if 'V' in [N,1,J]
        if morph_choice in list(morph[i]):
            #print(f"{morph_choice} in {morph_properies}")
            morph_attributes.append(1)
        else:
            #print(f"{morph_choice} not in {morph_properies}")
            morph_attributes.append(0)
           
                
    #M.append(morph_attributes + [transformed[i]])
    M.append(morph_attributes + [lemmas[i]] + [radical[i]] + [form[i]] + [transformed[i]])

M_n = np.array(M)

In [ ]:
# Study over duplicates of morphological attributes
mask = (M_n[:,3] == '1')
M_n[mask,-1]

In [8]:
import difflib
words = ['hello', 'Hallo', 'hi', 'house', 'key', 'screen', 'hallo', 'question', 'format']
difflib.get_close_matches('Hello', words)

['hello', 'Hallo', 'hallo']

In [9]:
language_path_test = "./task0-data/DEVELOPMENT-LANGUAGES/niger-congo/swa.tst"

f = open(language_path_test,encoding="utf-8")

lemmas_test = []
morph_test = []

for lines in f.readlines():
    a, b = lines.split("\t")
    lemmas_test.append(a)
    morph_test.append(b.split(";")[:-2])


df = pd.DataFrame(M_n,columns=list(morph_list)+["lemma"]+["radical"]+["form"]+["prefix_suffix"])

#test_set = df.sample(int(len(df)*0.05))

res = []

for i in tqdm(range(len(lemmas_test))):
    copy = df
    guess = lemmas_test[i]
    guess_morph = morph_test[i]

    for v in guess_morph:
        copy = copy[copy[v] == '1']

    for a in morph_list:
        if a not in guess_morph:
            copy = copy[copy[a] == '0']

    best_index = -1
    best_word = ''
    for index, row in copy.iterrows():
        if distance(guess, row['lemma']) < distance(guess, best_word):
            best_index = index
            best_word = row['lemma']
    #print("Guess",guess,"Best_word",best_word,"Best_index",lemmas[best_index])
    #print(copy.loc[best_index])
    #print(transformed[best_index].replace('[word]',guess.replace(radical[best_index].replace('[word]',''),'')),"->",form[i])
    #print(lemmas[i])
    res.append(transformed[best_index].replace('[word]',guess.replace(radical[best_index].replace('[word]',''),'')))

In [10]:
language_path_gold = "./task0-data/GOLD-TEST/swa.tst"

f = open(language_path_gold,encoding="utf-8")

lemmas_true = []
form_true = []
morph_true = []

for lines in f.readlines():
    a, b, c = lines.split("\t")
    lemmas_true.append(a)
    form_true.append(b)
    morph_true.append(c.split(";")[:-2])

def eval_form(gold, guess):
    """ compute average accuracy and edit distance for task 0 """
    correct, dist, total = 0., 0., 0.
    mean_lean = 0
    for i in range(len(gold)):
        mean_lean += len(gold[i])
        if gold[i] == guess[i]:
            correct += 1
        else:
            dist += distance(gold[i],guess[i])
        total += 1
    mean_lean /= len(gold)
    return (round(correct/total*100, 2), round(dist/total, 2), mean_lean)

print("Accuracy","Mean Distance", "Average lemma length")
eval_form(form_true, res)

Accuracy Mean Distance Average lemma length


(2.42, 2.3, 9.057142857142857)

----
Misc. testing

In [24]:
from sklearn.model_selection import train_test_split


#WIP : 06/02
guess =  'proidie'
guess_morph = ['V', 'IND', 'PL', '1']
df = pd.DataFrame(M_n,columns=list(morph_list)+["lemma"]+["radical"]+["form"]+["prefix_suffix"])

#test_set = df.sample(int(len(df)*0.05))
test_set = df.sample(1)
df = df.drop(test_set.index)

correct = 0
for i, _ in test_set.iterrows():
    copy = df
    guess = lemmas[i]
    guess_morph = morph[i]

    for v in guess_morph:
        copy = copy[copy[v] == '1']

    for a in morph_list:
        if a not in guess_morph:
            copy = copy[copy[a] == '0']

    best_index = -1
    best_word = ''
    for index, row in copy.iterrows():
        if distance(guess, row['lemma']) < distance(guess, best_word):
            best_index = index
            best_word = row['lemma']
    #print("Guess",guess,"Best_word",best_word,"Best_index",lemmas[best_index])
    #print(copy.loc[best_index])
    #print(transformed[best_index].replace('[word]',guess.replace(radical[best_index].replace('[word]',''),'')),"->",form[i])
    #print(lemmas[i])
    if transformed[best_index].replace('[word]',guess.replace(radical[best_index].replace('[word]',''),'')) == form[i]:
        correct += 1
    else:
        print("Erreur",transformed[best_index].replace('[word]',guess.replace(radical[best_index].replace('[word]',''),'')),"->",form[i])
        pass

correct / float(len(test_set)) * 100.0

Erreur bakwaˀan -> rukwaˀan


0.0

In [23]:
form_true[:10]

['nitakamilisha',
 'nilifagia',
 'wanaponda',
 'tungalimwita',
 'ameondoa',
 'tuhuchafya',
 'ninaficha',
 'ningeomba',
 'tumebofya',
 'mlifuata']

In [24]:
res[:10]

['utakamilisha',
 'nitafagia',
 'mnaponda',
 'wangalimwita',
 'umeondoa',
 'wahuchafya',
 'unaficha',
 'ningaliomba',
 'wamebofya',
 'walifuata']

-----

In [46]:
to_guess = lemmas_test[0]
to_guess_morph = morph_test[0]

df = pd.DataFrame(M_n,columns=list(morph_list)+["lemma"]+["radical"]+["form"]+["prefix_suffix"])
copy = df

for v in to_guess_morph:
    copy = copy[copy[v] == '1']

for a in morph_list:
    if a not in to_guess_morph:
        copy = copy[copy[a] == '0']

copy

,V,SG,PRF,PL,HAB,DEF,FIN,INDF,COND,IND,lemma,radical,form,prefix_suffix
3,1,1,0,0,0,0,1,0,0,1,tengeneza,[word],nitatengeneza,nita[word]
11,1,1,0,0,0,0,1,0,0,1,choma,[word],ulichoma,uli[word]
38,1,1,0,0,0,0,1,0,0,1,pika,[word],atapika,ata[word]
48,1,1,0,0,0,0,1,0,0,1,tafuta,[word],nilitafuta,nili[word]
54,1,1,0,0,0,0,1,0,0,1,funga,[word],utafunga,uta[word]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3343,1,1,0,0,0,0,1,0,0,1,panda,[word],utapanda,uta[word]
3347,1,1,0,0,0,0,1,0,0,1,kopa,[word],nitakopa,nita[word]
3353,1,1,0,0,0,0,1,0,0,1,lia,[word],nitalia,nita[word]
3359,1,1,0,0,0,0,1,0,0,1,ruka,[word],utaruka,uta[word]


In [50]:
import re

def to_regex(word):
    return "^" + word.replace("[word]",".+") + "$"

bool(re.match(r"^.+tua$",'unahtua'))
print(to_regex('[word]tiä'))

^.+tiä$


In [55]:
best_index = -1
best_word = ''
"""
for index, row in copy.iterrows():
    if distance(guess, row['lemma']) < distance(guess, best_word):
        best_index = index
        best_word = row['lemma']"""


for index, rad in enumerate(df["radical"].values.tolist()):
    if bool(re.match(to_regex(rad),to_guess)) and len(rad) >= len(best_match):
        #print(transformed[best_match_index], best_match)
        best_index, best_word = index, rad

print(transformed[best_match_index].replace('[word]',to_guess.replace(radical[best_match_index].replace('[word]',''),'')))

print("Original word :",to_guess)
print("Closest word found :",best_word)
print("Result :")
print(transformed[best_index].replace('[word]',guess.replace(radical[best_index].replace('[word]',''),'')))
print("Truth :")
print(form_true[0])

Original word : kamilisha
Closest word found : [word]
Result :
ningalisema
Truth :
nitakamilisha


In [165]:
df = pd.DataFrame(M_n,columns=list(morph_list)+["lemma"]+["radical"]+["form"]+["prefix_suffix"])
d2 = df[(df['lemma']=="anguka")]
d2

,IND,INDF,DEF,PL,PRF,HAB,COND,V,SG,FIN,lemma,radical,form,prefix_suffix
29,1,0,0,0,0,0,0,1,1,1,anguka,[word],ulianguka,uli[word]
172,0,0,0,1,0,0,1,1,0,1,anguka,[word],wangeanguka,wange[word]
246,1,0,0,1,1,0,0,1,0,1,anguka,[word],tumeanguka,tume[word]
262,1,0,0,0,0,1,0,1,1,1,anguka,[word],ahuanguka,ahu[word]
315,1,0,0,1,0,0,0,1,0,1,anguka,[word],tulianguka,tuli[word]
336,0,0,0,0,0,0,0,0,0,0,anguka,[word],kuanguka,ku[word]


In [159]:
d2 = df[(df['form']=="nihuandika")]
d2

,IND,INDF,DEF,PL,PRF,HAB,COND,V,SG,FIN,lemma,radical,form,prefix_suffix


In [65]:
to_guess = lemmas_test[0]
to_guess_morph = morph_test[0]

df = pd.DataFrame(M_n,columns=list(morph_list)+["lemma"]+["radical"]+["form"]+["prefix_suffix"])
copy = df

for v in to_guess_morph:
    copy = copy[copy[v] == '1']

for a in morph_list:
    if a not in to_guess_morph:
        copy = copy[copy[a] == '0']

best_match_index = -1
best_match = ''
for index, rad in enumerate(copy["radical"].values.tolist()):
    if bool(re.match(to_regex(rad),to_guess)) and len(rad) >= len(best_match):
        #print(transformed[best_match_index], best_match)
        best_match_index, best_match = index, rad

print("Original word :",to_guess)
print("Closest word found :",best_match)
print("Result :")
print(transformed[best_match_index].replace('[word]',guess.replace(radical[best_match_index].replace('[word]',''),'')))
print("Truth :")
print(form_true[0])


Original word : kamilisha
Closest word found : [word]
Result :
atasema
Truth :
nitakamilisha
